# 6B - Gaussian Mixture Model in a 2D data space

In the previous notebook, we introduced latent-reprensentation learning with auto-encoders. 
In the perspective of generative modeling, we have seen that auto-encoders can be used to learn a latent representation of the data, which can then be used to generate new samples by sampling from the latent space **appropriately** and decoding the samples back to the data space.
The major limitation of this approach is that the latent space is not guaranteed to be structured in a way that allows for meaningful sampling.

In this notebook, we will explore the concept of latent Gaussian Mixture Models (GMM) :
- First, we will introduce the Gaussian Mixture Model (GMM) and the concept of latent variables
- Second, we will investigate maximum likelihood estimation 
- Then we will study the Expectation-Maximization (EM) algorithm for the joint estimatation of the parameters of a GMM and the latent variables
- We will implement this algorithm in PyTorch and apply it to a simple dataset

Finally, we will conclude on the need to train a more sophisticated generative model.


julien rabin @ greyc.ensicaen.fr 2025

In [ ]:
import torch 
from torch import nn, Tensor

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

Data loader (same as the previous notebook)

In [ ]:
def sample_data(n: int, model = 'radial_gmm') -> Tensor:
    data_dim = 2  # Dimension of the data
    if model == 'moons':
        from sklearn.datasets import make_moons
        return Tensor(make_moons(n_samples=n, noise=0.05)[0])
    elif model == 'circles':
        from sklearn.datasets import make_circles
        return Tensor(make_circles(n_samples=n, noise=0.05, factor=0.5)[0])
    elif model == '2gmm':
        n_samples = n
        n = n//3
        X1 = torch.randn(n, data_dim) @ torch.tensor([[.05, -0.02],[-0.02, .4]]) + torch.tensor([[.0, 1.0]]).view(1, data_dim)
        X2 = torch.randn(n_samples - n, data_dim) @ torch.tensor([[.3, 0.05],[0.05, .05]]) + torch.tensor([[-1.0, 0.]]).view(1, data_dim)
        return torch.cat((X1,X2), dim=0)  # Concatenate the two sets of samples
    elif model == 'radial_gmm':
        K = 8
        n_samples = n
        samples_per_component = n_samples // K
        remainder = n_samples % K
        all_samples = []

        for k in range(K):
            radius = 3.
            # Angle for the mean on a circle
            theta = 2 * np.pi * k / K
            cs = np.cos(theta)
            sn = np.sin(theta)

            # Radial direction unit vector
            radial = torch.tensor([cs, sn], dtype=float)#.view(data_dim, 1)
            tangential = torch.tensor([-sn, cs], dtype=float)#.view(data_dim, 1)

            mean = radius * radial
            
            # Covariance matrix: elongated along radial direction
            cov = 0.3 * torch.outer(radial, radial).to(float) + 0.05 * torch.outer(tangential, tangential).to(float)

            # Generate samples
            n = samples_per_component + (remainder if k == K else 0)
            samples = torch.randn(n, data_dim, dtype=float) @ torch.linalg.cholesky(cov).to(float) + mean.to(float)
            all_samples.append(samples)

        return torch.cat(all_samples, dim=0).to(torch.float32)
    else:
        raise ValueError(f"Unknown model: {model}.")


## Part I : Gaussian Mixture Model (GMM)


### Course recap: model definition

The Gaussian Mixture Model (GMM) is a probabilistic model that assumes that the data is generated from a mixture of several Gaussian distributions, each with its own mean and covariance. The GMM is defined by the following parameters:
- $ K $: the number of Gaussian components in the mixture
- $ \pi_k $: the mixing coefficient for each component $k$, which represents the probability of selecting that component
- $\mu_k$: the mean of each component $k$
- $\Sigma_k$: the covariance matrix of each component $k$
The GMM can be expressed as a probability density function (PDF) as follows:
$$
p_\theta(x) = \sum_{k=1}^{K} \pi_k \mathcal{N}(x | \mu_k, \Sigma_k)
$$
where the model's parameters are $\theta = \{\pi_k, \mu_k,\Sigma_k \}_{k=1}^K$,
and
where $\mathcal{N}(x | \mu_k, \Sigma_k)$ is the Gaussian distribution with mean $\mu_k$ and covariance $\Sigma_k$.
$$
\mathcal{N}(x | \mu_k, \Sigma_k) = \frac{1}{(2\pi)^{d/2} |\Sigma_k|^{1/2}} \exp\left(-\frac{1}{2}(x - \mu_k)^T \Sigma_k^{-1} (x - \mu_k)\right)
$$

### latent representation
Let denote by $z$ the latent representation of the data $x$.
In the case of GMM, the latent representation is a categorical variable (generalized Bernoulli) that indicates which Gaussian component generated the data point $x$ : 
$$
    z \sim \text{Cat}(\pi_1, \ldots, \pi_K)
$$
such that the probability of $z=k$ is given by the mixing coefficient $\mathbb P(z=k) = \pi_k$.
The problem is generally that this latent representation is not directly observable, and we need to infer it directly from the data $x$ itself.

Using Bayesian inference, we can compute the posterior distribution of the latent variable $z$ given the data $x$:
$$
    p(z=k | x) 
    = \frac{p(x | z=k) p(z=k)}{p(x)} 
    = \frac{p(x | z=k) p(z=k)}{\sum_{j=1}^K p(x|z=j) p(z=j)} 
    = \frac{\pi_k \mathcal{N}(x | \mu_z, \Sigma_z)}{\sum_{j=1}^{K} \pi_j \mathcal{N}(x | \mu_j, \Sigma_j)}
$$


### Model optimization using Expectation-Maximization (EM) algorithm :

The goal is to maximize the likelihood of the data given the model parameters, that is for a set of $N$ data points $X = \{x_1, x_2, \ldots, x_N\}$, we want to maximize the log-likelihood: 
$$
    \mathcal{L}(\theta) 
    = \log p_\theta(X) 
    = \log \left( \prod_{n=1}^{N} p_\theta(x_n) \right) 
    = \sum_{n=1}^{N} \log p_\theta(x_n)
    = \sum_{n=1}^{N}  \log \left( \sum_{k=1}^{K} p(z_n=k) p_\theta(x_n | z_n=k) \right)
$$
where $p_\theta(x_n)$ is the probability density function of the GMM evaluated at the data point $x_n$.

Note that *if* the latent variable $z_n$ is observed, we can simply consider the conditional likelihood of the data given the latent variable, that is:
$$
    \log p_\theta(X|Z)
    = \sum_{n=1}^{N} \log p_\theta(x_n| z_n) 
$$
the maximization of which is straightforward using the MLE (Maximum Likelihood Estimation) for each component $k$ (parameters $\mu_k$ and $\Sigma_k$).
In practice, we can compute the expected value of such conditional log-likelihood to account for the uncertainty in the latent variable $z_n$ (parameters $\pi_k$):
$$
    \mathbb{E}_{Z|X,\theta}[\log p_\theta(X|Z)] 
    = \sum_{n=1}^{N} \sum_{k=1}^{K} p(z_n=k | x_n) \log p_\theta(x_n | z_n=k)
$$

This is the expected log-likelihood of the data given the latent variable, **which can be interpreted as a lower bound on the log-likelihood of the data** (see the course transcript for details !).
To optimize the GMM parameters $\theta$, we can use the Expectation-Maximization (EM) algorithm, which iteratively maximizes the expected log-likelihood of the data given the latent variable. The EM algorithm consists of two steps: the E-step and the M-step.


The GMM is typically trained using the Expectation-Maximization (EM) algorithm, which consists of two steps:
1. **Expectation step (E-step)**: estimates the latent variables $z_n$ for all data point $x_n$ using the current model parameters $\theta$. To do so, we compute the posterior probabilities of each component given the data, using the current parameters $\theta = \{\pi_k, \mu_k,\Sigma_k \}_{k=1}^K$.
We denote $\gamma_{k,n} = p(z=k|x_n)$ the posterior probability (density) of component $k$ given data point $n$, computed as (using Bayes' rule):
$$
\gamma_{k,n} 
    = p(z=k|x_n)
    = \frac{p(x_n|z=k) p(z=k)}{p(x_n)}
    = \frac{\pi_k \mathcal{N}(x_n | \mu_k, \Sigma_k)}{\sum_{j=1}^{K} \pi_j \mathcal{N}(x_n | \mu_j, \Sigma_j)}
$$
We denote $\Gamma = [\gamma_{k,n}]_{k,n}$ as the matrix of such posterior probabilities, and $X$ the matrix of data points ($N \times d$).

Note : this is a soft-assignment of the data points to the Gaussian components (i.e each data point belongs to multiple components with different probabilities). This is in contrast to hard assignment (as in K-means), where each data point is assigned to a single component.
On the one hand, this ensures that no component is empty which makes it numerically more stable, but on the other hand, it makes the optimization problem more complex as we need to compute the posterior probabilities for each data point and each component ($N \times K$). 

2. **Maximization step (M-step)**: Update the parameters $\theta$ of the model (mixing coefficients, means, and covariances) based on the posterior probabilities computed in the E-step, that is we use the maximum likelihood estimation (MLE) to update the parameters of each gaussian component $k$ separately:
$$
N_k = \sum_{n=1}^{N} \gamma_{k,n} = (\Gamma \mathbf{1})_k
$$
$$
\pi_k = \frac{1}{N} \sum_{n=1}^{N} \gamma_{k,n} = \frac{N_k}{N}
$$
$$
\mu_k = \frac{\sum_{n=1}^{N} \gamma_{k,n} x_n}{\sum_{n=1}^{N} \gamma_{k,n}}
= \frac{1}{N_k} (\Gamma X)_k
$$
$$
\Sigma_k = \frac{\sum_{n=1}^{N} \gamma_{k,n} (x_n - \mu_k)(x_n - \mu_k)^T}{\sum_{n=1}^{N} \gamma_{k,n}}
 = \frac{1}{N_k} \left( \Gamma \odot (X - \mu_k \mathbf{1}^T)(X - \mu_k \mathbf{1}^T)^T \right)_k
$$

Note: this is a non-convex optimization problem, and the EM algorithm is guaranteed to converge to a local maximum of the likelihood function. The convergence is guaranteed by the fact that the likelihood function is non-decreasing at each iteration of the EM algorithm.

Technical details : To compute $\gamma_{nk}$, we need to compute the probability density function of the Gaussian distribution for each component $k$ and each data point $x_n$, which can be done using the following formula:
$$
    \log \mathcal{N}(x_n | \mu_k, \Sigma_k)
    = -\frac{1}{2} \left( d \log(2\pi) + \log |\Sigma_k| + (x_n - \mu_k)^T \Sigma_k^{-1} (x_n - \mu_k) \right)
$$
In the 2D case, we can use the following formula to inverse the symmetric covariance matrix:
$$
\Sigma_k = \begin{pmatrix}
\sigma_{k,1}^2 & c_k \\
c_k & \sigma_{k,2}^2
\end{pmatrix}
,
\text{ and }
\Sigma_k^{-1} = \frac{1}{\sigma_{k,1}^2 \sigma_{k,2}^2 - c_k^2} 
\begin{pmatrix}
\sigma_{k,2}^2 & -c_k \\
-c_k & \sigma_{k,1}^2
\end{pmatrix}   
$$

where $\sigma_{k,1}$ and $\sigma_{k,2}$ are the standard deviations of the two dimensions of the Gaussian component $k$, and $\rho_k$ is the correlation coefficient between the two dimensions.

In [ ]:
def inv_2Dcov(cov: Tensor, eps : float = 1e-7) -> Tensor:
    """Compute the inverse of a 2D covariance matrix."""
    # cov : Tensor of shape (K_GMM, d, d)
    assert cov.shape[1]==2 and cov.shape[2]==2, "Covariance matrix must be 2x2"
    inv_cov = Tensor(cov.shape)  # Initialize the inverse covariance tensor
    det = Tensor(cov.shape[0])  # Initialize determinant tensor
    for k in range(cov.shape[0]):
        det[k] = (cov[k, 0, 0] * cov[k, 1, 1] - cov[k, 0, 1] * cov[k, 1, 0])
        #assert det[k] != 0, "Covariance matrix is singular, cannot compute inverse"
        if det[k] <= eps:
            print(f"Warning: Covariance matrix {k} is singular : {cov_k[k]}; reset to identity.")
            det[k] = 1
            cov[k,:,:] = torch.eye(2)  # Fallback to identity matrix if singular
        inv_cov[k,:,:] = Tensor([[cov[k, 1, 1], -cov[k, 0, 1]], [-cov[k, 1, 0], cov[k, 0, 0]]]) / det[k]
        # regularization
        #if eps is not None and eps > 0:
        #    inv_cov[k,:,:] += eps * torch.eye(2)
    return (inv_cov, det)

In [ ]:
def log_multiple_gauss(x: Tensor, mu: Tensor, cov: Tensor) -> Tensor:
    """Compute the log probability density function of multiple 2D Gaussians."""
    assert x.shape[1]==2, "Input x must be a 2D tensor with shape (n_samples, 2)"
    d = 2
    # x : Tensor of shape (n_samples, d)
    # mu : Tensor of shape (K_GMM, d)
    # cov : Tensor of shape (K_GMM, d, d)
    
    diff = x.permute(1,0).view(1,d,-1) - mu.view(-1, d, 1) # (K_GMM, d, n_samples)
    
    if True :
        inv_cov, det = inv_2Dcov(cov, eps=1e-7) # (K_GMM, d, d)
    else :
        inv_cov = torch.linalg.inv(cov)
        det = torch.linalg.det(cov)
    # Compute the exponent term
    exp_term = -0.5 * torch.sum(diff * (inv_cov @ diff), dim=1)  # (K_GMM, n_samples)
    
    norm_term = 0.5 * torch.log((2 * np.pi) ** d * det) # (K_GMM)
    
    return exp_term - norm_term.view(-1, 1)  # (K_GMM, n_samples)

select and adapt the number of Gaussian components $K=$ `K_GMM` based on the dataset used.

In [ ]:
torch.manual_seed(42)  # For reproducibility

K_GMM = ...  # Number of Gaussian components

data_dim = 2  # Dimension of the data
assert data_dim == 2, "This implementation only supports 2D data."

# GMM parameters definition and initialization
mu_k = torch.randn(K_GMM, data_dim)  # Randomly initialized means (independent from data)
cov_k = torch.stack([torch.eye(data_dim) for _ in range(K_GMM)])  # Identity matrices as initial covariances
pi_k = torch.ones(K_GMM,1) / K_GMM  # Equal weights for each component
print(f"{mu_k.shape=}, {cov_k.shape=}, {pi_k.shape=}")

In [ ]:
n_samples = 1_000  # Number of samples to generate
dataset_name = '2gmm' # moons, circles, 2gmm, radial_gmm
X = sample_data(n_samples, model=dataset_name).detach()  # Generate synthetic data (n_samples, d)

# compute the likelihood of the training / test data given the GMM parameters
LL_train = log_multiple_gauss(X, mu_k, cov_k) # (K_GMM, n_samples)
LL_train = torch.log(torch.sum(pi_k * torch.exp(LL_train), dim=0)) # (1, n_samples)  
print(f"Log-likelihood of the TRAIN data with random GMM parameters: {torch.mean(LL_train).item()}")

n_iter = 100

for it in tqdm(range(n_iter)):
    if torch.any(torch.isnan(mu_k)) or torch.any(torch.isnan(cov_k)) or torch.any(torch.isnan(pi_k)):
        print(f"NaN detected in parameters at iteration {it}, stopping.")
        break
    #for k in range(K_GMM):
    #    if torch.allclose(cov_k[k], cov_k[k].T, rtol=1e-6,  atol=1e-6) :
    #        print(f"Covariance matrix {k} is not symmetric: {cov_k[k]} !")
    
    ## E-step
    gamma = log_multiple_gauss(X,mu_k,cov_k) # shape (K_GMM, n_samples)
    if torch.any(torch.isnan(gamma)):
        print(f"NaN detected in 'log_multiple_gauss' at iteration {it}, stopping.")
        break
    
    gamma = torch.exp(gamma)  # Convert log probabilities to probabilities
    gamma *= pi_k.view(-1, 1)  # Multiply by the weights
    gamma_sum = torch.sum(gamma, dim=0, keepdim=True)  # Sum over components for each sample
    
    idx = gamma_sum[0,:] > 0  # detect empty components : this should not happen with infinite support gaussians exept for numerical issues (e.g. singular covariance)
    if idx.sum().item() < n_samples :
        print(f"Warning: division by zero ...")
        break
    gamma /= gamma_sum # Normalize to get probabilities
    
    ## M-steps
    N_k = gamma.sum(dim=1, keepdim=True) # (K_GMM, 1)
    pi_k = N_k / n_samples  # Update weights
    mu_k = (gamma @ X) / N_k
    diff = X.view(1, n_samples, -1) - mu_k.view(K_GMM, 1, -1)  # (K_GMM, n_samples, d) 
    cov_k = diff.permute(0, 2, 1) @ (gamma.view(K_GMM,n_samples,1) * diff)  # (K_GMM, d, n_samples) @ (K_GMM, n_samples, d) -> (K_GMM, d, d)
    cov_k = cov_k / N_k.view(-1, 1, 1)  # (K_GMM, d, d)
    
    # reset small components
    idx = N_k < 0.01 * n_samples / K_GMM # arbitrary threshold 
    idx = [i for i, val in enumerate(idx) if val]
    if len(idx) == 0:
        continue  # No small components found, continue to next iteration
    print(f"{it=} Warning: small components found {idx=} with N_k[:] = {gamma_sum[0,idx]}, resetting them.")
    for k in idx:
        print(f"Resetting empty component {k}")
        i = torch.randperm(n_samples)[0] # Randomly select K_GMM points from the data
        mu_k [k,:] = X[i,:]  # Initialize means with random points from the data
        cov_k[k,:] = torch.eye(data_dim)  # Reset empty components to identity matrix 
        pi_k [k,:] = 1.0 / K_GMM  # Reset weights to equal distribution



In [ ]:
print(f"Final parameters after {n_iter} iterations:")
print(f"{mu_k=}\n{cov_k=}\n{pi_k=}")

In [ ]:
# sanity check: compare fitting using sklearn 'GaussianMixture'
if False :
    from sklearn.mixture import GaussianMixture
    gmm = GaussianMixture(n_components=K_GMM, covariance_type='full', max_iter=n_iter, random_state=42)
    gmm.fit(X.numpy())  # Fit the GMM to the data
    print(f"Sklearn GMM parameters:")
    print(f"{gmm.means_}\n{gmm.covariances_}\n{gmm.weights_}")

    if True : # use these parameters
      mu_k = Tensor(gmm.means_)
      cov_k = Tensor(gmm.covariances_)
      pi_k = Tensor(gmm.weights_)

In [ ]:
def sample_multivariate_normal(n : int, mu: Tensor, cov_sqrt: Tensor) -> Tensor:
    """Samples from a multivariate normal distribution."""
    assert n> 0, "Number of samples must be positive"
    d = mu.shape[0]  # Dimension of the data
    z = torch.randn(n,d)  # Sample from standard normal distribution
    z = mu.view(1,d) + z @ cov_sqrt  # Return the sample from the multivariate normal distribution
    return z # shape (n, d)

def compute_cov_sqrt(cov: Tensor) -> Tensor:
    """Compute the square root of a covariance matrix using Cholesky decomposition."""
    # cov : Tensor of shape (K_GMM, d, d)
    cov_sqrt = torch.zeros_like(cov)  # Initialize the square root tensor
    for k in range(cov.shape[0]):
        cov_sqrt[k] = torch.linalg.cholesky(cov[k]).T  # Compute the Cholesky decomposition. Warning, we need the transpose
    return cov_sqrt

# Sampling with the learned GMM parameters
def sample_gmm(n_samples: int, mu: Tensor, cov: Tensor, pi: Tensor) -> Tensor:
    """Sample from a Gaussian Mixture Model."""
    K = mu.shape[0]  # Number of components
    
    cov_sqrt = compute_cov_sqrt(cov)  # Compute the square root of the covariance matrices
    
    if K == 1: # no mixture
        samples = sample_multivariate_normal(n_samples, mu_k[0], compute_cov_sqrt(cov_k)[0])
        labels = torch.zeros(n_samples, dtype=torch.long)
        return samples, labels
    
    # for faster sampling, we draw directly the number of samples for each component
    N_k = torch.multinomial(pi.squeeze(), n_samples, replacement=True)  # Sample component indices based on the weights
    N_k = torch.bincount(N_k, minlength=K)  # Count the number of samples for each component
    assert N_k.sum() == n_samples, "Total number of samples must match n_samples"
    #print(f"Sampled {N_k} samples from each component")
    
    samples, labels = [], []
    for k in range(K):  # Sample a component index
        if N_k[k] == 0:
            continue
        sample = sample_multivariate_normal(N_k[k], mu[k], cov_sqrt[k])  # Sample from the selected component
        samples.append(sample)
        labels.append(torch.full((N_k[k],), k, dtype=torch.long))
    samples = torch.cat(samples, dim=0)  # (n,d)
    labels = torch.cat(labels, dim=0)  # (n,)
    # random permutation of the samples
    idx = torch.randperm(n_samples)  # Random permutation of the samples
    samples = samples[idx,:]
    labels = labels[idx]
    return samples, labels

In [ ]:
def define_colors(x_data: np.ndarray ) -> np.ndarray:
    cmap = plt.get_cmap("rainbow")
    values = x_data[:, 0]  # Use the first dimension for coloring
    normalized = (values - values.min()) / (values.max() - values.min())
    colors = cmap(normalized)
    return colors[:, :3]

In [ ]:
x_train = X.detach()
x_model, labels = sample_gmm(n_samples, mu_k, cov_k, pi_k)

fig, axes = plt.subplots(1,3,figsize=(15, 5), sharex=True, sharey=True)
axes[0].scatter(x_train[:, 0], x_train[:, 1], s=10, c='C0', label='Training Data Samples')


colors = define_colors(labels.view(-1,1).detach().numpy())
axes[1].scatter(x_model[:, 0].detach(), x_model[:, 1].detach(), s=10, c=colors, label='Generated Samples')

# superpositions
axes[2].scatter(x_model [:, 0], x_model [:, 1], s=100, c='C1', alpha=0.1, label='Fake Data Samples')
axes[2].scatter(x_train [:, 0], x_train [:, 1], s=100, c='C2', alpha=0.1, label='Train Data Samples')


axes[0].set_title('Training Data Samples')
axes[1].set_title('Generated Samples')
axes[2].set_title('superposition')
fig.suptitle("GMM Sampling Comparison", fontsize=24)
plt.tight_layout()
plt.show()


In [ ]:
# compute the likelihood of the training / test data given the GMM parameters
LL_train = log_multiple_gauss(x_train, mu_k, cov_k) # (K_GMM, n_samples)
LL_train = torch.log(torch.sum(pi_k * torch.exp(LL_train), dim=0)) # (1, n_samples)  
print(f"Avg. Log-likelihood of the TRAIN data given the GMM parameters: {torch.mean(LL_train).item()}")

x_test = sample_data(n_samples, model=dataset_name).detach()
LL_test = log_multiple_gauss(x_test, mu_k, cov_k) # (K_GMM, n_samples)
LL_test = torch.log(torch.sum(pi_k * torch.exp(LL_test), dim=0)) # (1, n_samples)  
print(f"Avg. Log-likelihood of the TEST data given the GMM parameters: {torch.mean(LL_test).item()}")

#with unrelated data :
x_outlier = torch.randn(n_samples, data_dim) * 1.0 + 10  # Generate random outlier data
LL_outlier = log_multiple_gauss(x_outlier, mu_k, cov_k) # (K_GMM, n_samples)
LL_outlier = torch.log(torch.sum(pi_k * torch.exp(LL_outlier), dim=0)) # (1, n_samples)
print(f"Avg. Log-likelihood of the OUTLIER data given the GMM parameters: {torch.mean(LL_outlier).item()}")

### Exercice
Number of components:
Experiment with the number of components $K$ of the GMM. 
- What happens if $K$ is too small or too large compared to the true number of clusters in the data ?
- What is the risk when $K$ becomes too large ? with respect to what ?
- What is the risk when $K$ becomes too small ? 

Likelihood evaluation: 
- compare the log-likelihood of the training data with random vs fitted GMM parameters
- compare the log-likelihood of the training and test data given the trained/fitted GMM parameters
- What happens with outlier data ?
- In conclusion, what is the major interest of using a GMM ?



## Part II : Latent Gaussian Mixture Model using Auto-Encoder


In [ ]:
# see the next notebook 6C for the latent GMM training !